# Delta Table Examples

Examples of creating and deleting delta tables and schemas.

In [0]:
# Replace cooplakehoues with the name of your storage account.
storage_end_point = "cooplakehouse.dfs.core.windows.net"

# Use the name of the secret scope that you set up along with the name of the secret in the key vault containing the storage account access key.
my_scope = "coop-lakehouse-scope"
my_key = "cooplakehouse-key"

spark.conf.set(
    "fs.azure.account.key." + storage_end_point,
    dbutils.secrets.get(scope=my_scope, key=my_key)
)

# Replace the container name (lakehouseblogstore) and storage account name (cooplakehouse) in the uri.
uri = "abfss://lakehouseblobstore@cooplakehouse.dfs.core.windows.net/TrainingFiles/"

In [0]:
%sql
-- Create a new schema for this example.  
CREATE SCHEMA IF NOT EXISTS grades

In [0]:
%sql
-- Example for dropping the table if it exists.
DROP TABLE IF EXISTS grades.dim_student_info

-- For an external table, you will need to manually delete the Parquet and delta metadata files in the storage account.


In [0]:
%sql
DROP SCHEMA IF EXISTS grades CASCADE

In [0]:
%sql
-- Create the table using an external location.
CREATE TABLE IF NOT EXISTS grades.dim_student_info
USING DELTA LOCATION "abfss://lakehouseblobstore@cooplakehouse.dfs.core.windows.net/TrainingFiles/DIM_StudentInfoTable"

In [0]:
# Populate the data.  Read the student info CSV file and save it.
from pyspark.sql.types import StructType, StructField, StringType, IntegerType, FloatType

file_schema = StructType([StructField("StudentID", IntegerType(), True), \
                    StructField("Major", StringType(), True), \
                    StructField("HomeState", StringType(), True) ])
students_df = spark.read.options(delimiter=',', header=True).schema(file_schema).csv(uri+"StudentInfo.csv")

display(students_df)


StudentID,Major,HomeState
1,Computer Science,North Dakota
2,Computer Science,Minnesota
3,Computer Science,Minnesota
4,Computer Science,North Dakota
5,Computer Science,Montana
6,Computer Science,Minnesota
7,Computer Science,Minnesota
8,Computer Engineering,Montana
9,Other,North Dakota
10,Computer Science,North Dakota


In [0]:
# Save using delta lake syntax
students_df.write.format("delta").mode("overwrite").option("mergeSchema", "true").save(uri + "DIM_StudentInfoTable")

In [0]:
%sql
-- Get data
SELECT * FROM grades.dim_student_info WHERE HomeState='Minnesota'


StudentID,Major,HomeState
2,Computer Science,Minnesota
3,Computer Science,Minnesota
6,Computer Science,Minnesota
7,Computer Science,Minnesota
12,Other,Minnesota
15,Computer Science,Minnesota
16,Computer Science,Minnesota
19,Computer Science,Minnesota
26,Computer Engineering,Minnesota
27,Computer Science,Minnesota


In [0]:

my_df = _sqldf

display(my_df)

StudentID,Major,HomeState
2,Computer Science,Minnesota
3,Computer Science,Minnesota
6,Computer Science,Minnesota
7,Computer Science,Minnesota
12,Other,Minnesota
15,Computer Science,Minnesota
16,Computer Science,Minnesota
19,Computer Science,Minnesota
26,Computer Engineering,Minnesota
27,Computer Science,Minnesota
